In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torch import nn

!pip install transformers
!pip install sentencepiece
from transformers import XLNetTokenizer, XLNetModel

!pip install syntok
import pandas as pd
import numpy as np
import regex as re
import syntok.segmenter as segmenter

from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 25.4 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 6.6 MB 58.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 36.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !pip3 install syntok
# !pip install sentencepiece
# import os
# import re
# import numpy as np 
# import pandas as pd
# import torch
# from transformers import XLNetTokenizer, XLNetModel
# from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
# from torch import nn, optim
# from IPython.display import FileLink, FileLinks
# from torch.utils.data import Dataset, DataLoader
# import pickle
# from tqdm import tqdm
# import syntok.segmenter as segmenter

In [4]:
## Run
class cue_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.xlmodel = XLNetModel.from_pretrained('xlnet-base-cased')
        self.lin = nn.Linear(768,4)
        
    def forward(self,x,att):
        xl=self.xlmodel(x,attention_mask=att)[0]
        xl = xl.view(-1,xl.shape[2])
        lin = self.lin(xl)
        return (lin)

## Run
class scoperes_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.xlmodel = XLNetModel.from_pretrained('xlnet-base-cased')
        self.lin = nn.Linear(768,2)
        
    def forward(self,x,att):
        xl=self.xlmodel(x,attention_mask=att)[0]
        xl = xl.view(-1,xl.shape[2])
        lin = self.lin(xl)
        return (lin)

In [5]:
## Run
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
## Run
model = torch.load('/content/drive/MyDrive/ML/innovation_lab/Hedge cues and span detection model/cue_only_model_ep3.pt',map_location=torch.device(device))
# model.eval()

In [7]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
# s = 'I assume that it might do it.'
# encodings = tokenizer.encode_plus(s,add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True)

In [8]:
path2018='/content/drive/MyDrive/ML/innovation_lab/Annotated_Review_18'

In [10]:
data=pd.read_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Review_Dataset_temp.csv')
data

,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,0.008333,4,2,"{'ABS': 0, 'INT': 0, 'RWK': 2, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 3, 'IMP': 0, 'CMP': 0, 'PNF'...",0.286886,0.223556
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,0.005000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.361318,0.225954
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,0.014628,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 5, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 4, 'PNF'...",0.577550,0.459101
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,0.002242,11,7,"{'ABS': 0, 'INT': 2, 'RWK': 10, 'PDI': 3, 'DAT...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 5, 'PNF'...",0.789854,0.785466
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,0.000000,8,3,"{'ABS': 0, 'INT': 2, 'RWK': 1, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.572586,0.336443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,0.006623,10,4,"{'ABS': 2, 'INT': 3, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.717566,0.453774
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,0.003106,8,7,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 2, 'CMP': 4, 'PNF'...",0.573514,0.780257
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,0.006231,9,5,"{'ABS': 0, 'INT': 1, 'RWK': 2, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 2, 'PNF'...",0.646486,0.561458
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,0.014440,9,6,"{'ABS': 0, 'INT': 1, 'RWK': 4, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 1, 'IMP': 0, 'CMP': 3, 'PNF'...",0.644999,0.671141


In [11]:
data.shape

(1322, 17)

In [12]:
sent_outliers = ['"','Pros : 1 .','( 2017 ) .','Cons : 1 .','Sec .','Comments : 1 .','( 2016 ) .','Minor comments : 1 .','" 1 .','10 .','12 .','" SUMMARY .','( 4 ) .','Eq .','Weaknesses : 1 .','Detailed comments : 1 .','- Page 4 .','( 2015 ) .','Clarity : 1 .','- p .','11 .','13 .','( 2011 ) .','" Pros : 1 .','Pros ------ 1 .','Strengths : 1 .','Int .','EMNLP .','3b .','( 2008 ) .','In Eq .','C2 .','( 7 ) ?','MLHC 2017 .','In CVPR , 2016 .','In NIPS , 2017 .','( 2 ) .','- Page 8 .']
sent=[]
s=0

for i in range(data.shape[0]):
  # text=data.loc[i,'review'][2:].split("\', \'")[0].replace("\\\'" , "").replace("#","")
  # print('\n',i,type(data.loc[i,'review']),data.loc[i,'review'],sep='\n')
  if data.loc[i,'review'] is np.nan:
    text=''
  else:
    text=(data.loc[i,'review']).replace("\\\'" , "").replace("#","")
  text = re.sub(r'\\u201.', '', text)
  text = re.sub(r'\\n\\n', '\n', text)
  text = re.sub(r'\\n', ' ', text)
  text = re.sub(r'\\ n', ' ', text)
  text = re.sub("\'", "", text)
  text = re.sub(r"\\","", text)

  for paragraph in segmenter.process(text):
    no=0
    for sentence in paragraph:
      d={}
      sen = ' '.join([j.value for j in sentence])
      if(sen in sent_outliers):
        continue
      no+=1
      d['sentence'] = ' '.join([j.value for j in sentence])
      d['review_id'] = i+1
      d['sentence_id'] = no
      sent.append(d)

In [13]:
print(type(sent),len(sent),sent[0],sep='\n')

<class 'list'>
22702
{'sentence': 'The idea of using cross task transfer performance to do task clustering is not new .', 'review_id': 1, 'sentence_id': 1}


In [14]:
## Run
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
token_list=[]
for sen in tqdm(sent):
    d={}
    # inp_ids = tokenizer.encode_plus(sen['sentence'],add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True)
    inp_ids=tokenizer(sen['sentence'],add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True)
    d['input_ids'] = inp_ids['input_ids']
    d['attention_mask'] = inp_ids['attention_mask']
    d['sentence'] = sen['sentence']
    d['review_id'] = sen['review_id']
    d['sentence_id'] = sen['sentence_id']
    token_list.append(d)
# token_list = [tokenizer.encode_plus(i['sentence'],add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True) for i in sent]

100%|██████████| 22702/22702 [00:07<00:00, 3106.27it/s]


In [15]:
print(type(token_list),len(token_list),token_list[0],sep='\n')

<class 'list'>
22702
{'input_ids': tensor([[  32,  875,   20,  381, 1650, 2578, 2457,  922,   22,  112, 2578, 9774,
           56,   27,   50,  109,   17,    9,    4,    3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'sentence': 'The idea of using cross task transfer performance to do task clustering is not new .', 'review_id': 1, 'sentence_id': 1}


In [16]:
## run
cues_list=[]
max_len=100
model=model.eval()
with torch.no_grad():
    for t in tqdm(token_list):
        inp = t['input_ids'].to(device)
        att = t['attention_mask'].to(device)
        revid = t['review_id']
        sentid = t['sentence_id']
#         print(inp)
#         break
        pred = torch.max(model(inp,att),dim=1)[1]
        hedges = [tokenizer.decode(i) for i,j in zip(inp[0],pred) if (j==1 or j==2)]
        cues_list.append({'review_id':revid,'sentence_id':sentid,'pred':pred,'hedges':hedges})
#         print(pred)
#         print([tokenizer.decode(i) for i in inp[0]])
#         print(hedges)
#         print('*'*100)

100%|██████████| 22702/22702 [07:48<00:00, 48.48it/s]


In [17]:
len(cues_list)

22702

In [18]:
cues_list[:10]

[{'hedges': ['not', 'new'],
  'pred': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0],
         device='cuda:0'),
  'review_id': 1,
  'sentence_id': 1},
 {'hedges': [],
  'pred': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
  'review_id': 1,
  'sentence_id': 2},
 {'hedges': [],
  'pred': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
  'review_id': 1,
  'sentence_id': 3},
 {'hedges': ['indicates', 'that'],
  'pred': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         device='cuda:0'),
  'review_id': 1,
  'sentence_id': 4},
 {'hedges': [],
  'pred': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
## Run
scopeinp_list=[]
for i,cue_pred in tqdm(zip(token_list,cues_list)):
    pred = cue_pred['pred'].tolist()
    revid = i['review_id']
    sentid = i['sentence_id']
    raw_sent = i['sentence']
    idx=-1
    if(sum(pred)==0):
        scopeinp_list.append({'review_id':revid,'sentence_id':sentid,'raw_sentence':raw_sent,'input_ids':i['input_ids'],'attention_mask':i['attention_mask'],'hedge_cue':None})
    while(True):
        try:
            ip = i['input_ids'].view(-1).tolist()
            at = i['attention_mask'].view(-1).tolist()
            
            idx = pred.index(1,idx+1)
            hedge_cue = tokenizer.decode(ip[idx])
#             print('old ip :')
#             print(ip,'\n')
            ip.insert(idx, 1)
            at.append(1)
            ip = torch.tensor(ip).view(1,-1)
            at = torch.tensor(at).view(1,-1)
            scopeinp_list.append({'review_id':revid,'sentence_id':sentid,'raw_sentence':raw_sent,'input_ids':ip,'attention_mask':at,'hedge_cue':hedge_cue})
#             print(ip,'\n')
        except:
            break
    idx=-1
    while(True):
        try:
            ip = i['input_ids'].view(-1).tolist()
            at = i['attention_mask'].view(-1).tolist()

            idx = pred.index(2,idx+1)
            
#             pred.insert(idx, 2)
            ip.insert(idx, 2)
            at.append(1)
            ip_idx = idx+1
            
            hedge_cue = tokenizer.decode(ip[ip_idx])
#             print(tokenizer.decode(ip),'\n')
#             print(len(pred))
#             print(len(ip))
#             print(hedge_cue,'\n')
            while(idx+1<len(pred) and pred[idx+1]==2):
                hedge_cue += ' '+tokenizer.decode(ip[ip_idx+1])
                idx += 1
                ip_idx += 1
                
            ip = torch.tensor(ip).view(1,-1)
            at = torch.tensor(at).view(1,-1)
#             print(cue_pred['pred'].tolist(),'\n')
#             print(ip,'\n')
#             print(hedge_cue)
#             print(cue_pred['hedges'])
#             print('\n')
            scopeinp_list.append({'review_id':revid,'sentence_id':sentid,'raw_sentence':raw_sent,'input_ids':ip,'attention_mask':at,'hedge_cue':hedge_cue})
            
            
        except:
            break

22702it [00:02, 9831.54it/s] 


In [20]:
len(scopeinp_list)

24810

In [21]:
## Run
scope_model = torch.load('/content/drive/MyDrive/ML/innovation_lab/Hedge cues and span detection model/scoperes_model_ep3.pt',map_location=torch.device(device))
# scope_model.eval()

In [22]:
## Run
scope_list=[]
max_len=100
scope_model=scope_model.eval()
with torch.no_grad():
    for t in tqdm(scopeinp_list):
        inp = t['input_ids'].to(device)
        att = t['attention_mask'].to(device)
        revid = t['review_id']
        sentid = t['sentence_id']
        hedge = t['hedge_cue']
        raw_sent = t['raw_sentence']
        
        if(hedge==None):
            scope_list.append({'Review_id':revid,'Sentence_id':sentid,'Raw Sentence':raw_sent,'Hedged Sentence':tokenizer.decode(inp[0][:-2]),'Hedge':None,'Span':None})
            continue
            
        sc_pred = torch.max(scope_model(inp,att),dim=1)[1]
        sc = sc_pred.tolist()

        if(sum(sc)==0):
            scope_list.append({'Review_id':revid,'Sentence_id':sentid,'Raw Sentence':raw_sent,'Hedged Sentence':tokenizer.decode(inp[0][:-2]),'Hedge':hedge,'Span':None})
            continue
            
        inpsent = inp[0].tolist()
        scope_startid = sc.index(1)
        scope_endid = sc.index(0,scope_startid+1)
        
        inpsent = inpsent[:scope_endid]+[17,7967,17]+inpsent[scope_endid:]
        inpsent = inpsent[:scope_startid]+[17,7967,17]+inpsent[scope_startid:]
        inpsent = tokenizer.decode(inpsent[:-2])
        inpsent = inpsent.replace("#", "<span>",1)
        inpsent = inpsent.replace("#", "</span>",1)
        
        if(inpsent.find('<s>')>-1):
            inpsent = inpsent.replace("<s>", " <s> ",1)
            hedge_end_idx = inpsent.find(hedge)+len(hedge)
            inpsent = inpsent[:hedge_end_idx]+' <s> '+ inpsent[hedge_end_idx:]

        else:
            inpsent = inpsent.replace("</s>", " </s> ",1)
            hedge_end_idx = inpsent.find(hedge)+len(hedge)
            inpsent = inpsent[:hedge_end_idx]+' </s> '+ inpsent[hedge_end_idx:]
        
        scopetag_startid = inpsent.find('<span>')+len('<span>')
        scopetag_endid = inpsent.find('</span>')
        scope = inpsent[scopetag_startid:scopetag_endid]
        
        scope_list.append({'Review_id':revid,'Sentence_id':sentid,'Raw Sentence':raw_sent,'Hedged Sentence':inpsent,'Hedge':hedge,'Span':scope})

100%|██████████| 24810/24810 [02:41<00:00, 153.76it/s]


In [23]:
## Run
df = pd.DataFrame(scope_list)
df['Span']

0          The idea of using cross task transfer perfor...
1                                                     None
2                                                     None
3          </s>  indicates that </s>  by changing the s...
4                                                     None
                               ...                        
24805                                                 None
24806                                                 None
24807                                                 None
24808      <s>  possible <s>  to encode and decode tree...
24809      <s>  should <s>  present a comparison with s...
Name: Span, Length: 24810, dtype: object

In [24]:
df

,Review_id,Sentence_id,Raw Sentence,Hedged Sentence,Hedge,Span
0,1,1,The idea of using cross task transfer performa...,<span> The idea of using cross task transfer ...,not new,The idea of using cross task transfer perfor...
1,1,2,"Please refer to the paper "" Discovering struct...","Please refer to the paper "" Discovering struct...",None,None
2,1,3,One issue of the use of cross task transfer pe...,One issue of the use of cross task transfer pe...,None,None
3,1,4,"For example , in binary classification tasks ,...","For example, in binary classification tasks, a...",indicates that,</s> indicates that </s> by changing the s...
4,1,5,So the use of cross task transfer performance ...,So the use of cross task transfer performance ...,None,None
...,...,...,...,...,...,...
24805,1322,1,This paper presents a model to encode and deco...,This paper presents a model to encode and deco...,None,None
24806,1322,2,This is not the first attempt of doing these e...,This is not the first attempt of doing these e...,None,None
24807,1322,3,"However , there is not a comparative evalution...","However, there is not a comparative evalution ...",None,None
24808,1322,4,"In fact , it has been demonstrated that it is ...","In fact, it has been demonstrated that it is <...",possible,<s> possible <s> to encode and decode tree...


In [25]:
count=0
for i in range(df.shape[0]):
  if(df.loc[i,'Hedge']==None):
    count+=1
count

16333

In [26]:
df.dropna(subset=['Hedge'],inplace=True)
df

,Review_id,Sentence_id,Raw Sentence,Hedged Sentence,Hedge,Span
0,1,1,The idea of using cross task transfer performa...,<span> The idea of using cross task transfer ...,not new,The idea of using cross task transfer perfor...
3,1,4,"For example , in binary classification tasks ,...","For example, in binary classification tasks, a...",indicates that,</s> indicates that </s> by changing the s...
7,1,8,Appendix A seems obvious but it cannot prove t...,<span> Appendix A <s> seems <s> obvious </s...,seems,Appendix A <s> seems <s> obvious
9,1,10,I dont know whether the low rank structure doe...,I dont know <span> <s> whether <s> the low ...,whether,<s> whether <s> the low rank structure doe...
10,1,10,I dont know whether the low rank structure doe...,I <span> </s> </s> dont know whether the lo...,don t know,</s> </s> dont know whether the low rank s...
...,...,...,...,...,...,...
24801,1321,15,- For the objective function defined in the pa...,- For the objective function defined in the pa...,may,<s> may <s> be hard to balance the structu...
24802,1321,15,- For the objective function defined in the pa...,- For the objective function defined in the pa...,may,<s> may not be even useful in real tasks ( ...
24803,1321,16,Earlier work on tree kernels ( in terms of def...,<span> Earlier work on tree kernels ( in term...,may,Earlier work on tree kernels ( in terms of d...
24808,1322,4,"In fact , it has been demonstrated that it is ...","In fact, it has been demonstrated that it is <...",possible,<s> possible <s> to encode and decode tree...


In [27]:
df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/review_hedge_sent.csv',index=False)

In [28]:
data['hedge_score']=0

In [29]:
for i in tqdm(range(data.shape[0])):
  new_data=df[df['Review_id']==i+1].copy(deep=True).reset_index(drop=True)
  # print(new_data.shape)
  # new_data.drop(subset=['Hedge'],inplace=True)
  if data.loc[i,'review'] is np.nan:
    review=''
  else:
    review=data.loc[i,'review'].replace("\\\'" , "").replace("#","")
  # print(review)
  text=""
  for j in range(new_data.shape[0]):
    text+=" "+new_data.loc[j,'Hedge']
  hedge_score=len(text.split(" "))/len(review.split(" "))
  data.loc[i,'hedge_score']=hedge_score
  # print(hedge_score)
  # if(i==3):
  #   break

100%|██████████| 1322/1322 [00:01<00:00, 974.20it/s] 


In [30]:
data[data['hedge_score']==0].shape

(0, 18)

In [31]:
data

,review_id,meta_decision,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,hedge_score
0,ICLR2018-B11bwYgfM-R1,NaN,The idea of using cross-task transfer performa...,12,242,20.166667,5.223176,121,2,240,0.008333,4,2,"{'ABS': 0, 'INT': 0, 'RWK': 2, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 3, 'IMP': 0, 'CMP': 0, 'PNF'...",0.286886,0.223556,0.053719
1,ICLR2018-B11bwYgfM-R2,NaN,This paper proposes a method for multitask and...,19,402,25.125000,5.311653,158,2,400,0.005000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.361318,0.225954,0.007389
2,ICLR2018-B11bwYgfM-R3,NaN,The authors propose techniques for multitask a...,35,763,19.564103,4.860955,278,11,752,0.014628,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 5, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 4, 'PNF'...",0.577550,0.459101,0.037859
3,ICLR2018-B12Js_yRb-R1,Accept,Summary: - This paper proposes a hand-designe...,30,447,14.900000,5.240196,210,1,446,0.002242,11,7,"{'ABS': 0, 'INT': 2, 'RWK': 10, 'PDI': 3, 'DAT...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 5, 'PNF'...",0.789854,0.785466,0.014925
4,ICLR2018-B12Js_yRb-R2,Accept,Summary - This paper mainly focuses on a coun...,12,243,20.250000,5.757447,125,0,243,0.000000,8,3,"{'ABS': 0, 'INT': 2, 'RWK': 1, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.572586,0.336443,0.023256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,ICLR2018-SysEexbRb-R2,Accept,This paper studies the critical points of shal...,23,456,20.727273,5.265700,194,3,453,0.006623,10,4,"{'ABS': 2, 'INT': 3, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 0, 'IMP': 0, 'CMP': 1, 'PNF'...",0.717566,0.453774,0.036403
1318,ICLR2018-SysEexbRb-R3,Accept,This paper mainly focuses on the square loss f...,16,323,14.681818,5.738562,137,1,322,0.003106,8,7,"{'ABS': 0, 'INT': 1, 'RWK': 3, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 2, 'CMP': 4, 'PNF'...",0.573514,0.780257,0.015198
1319,ICLR2018-Syt0r4bRZ-R1,Reject,Summary: the paper proposes a tree2tree archit...,20,323,14.681818,5.124590,167,2,321,0.006231,9,5,"{'ABS': 0, 'INT': 1, 'RWK': 2, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 2, 'PNF'...",0.646486,0.561458,0.023952
1320,ICLR2018-Syt0r4bRZ-R2,Reject,This paper proposes a tree-to-tree model aimin...,19,281,15.611111,4.949458,139,4,277,0.014440,9,6,"{'ABS': 0, 'INT': 1, 'RWK': 4, 'PDI': 1, 'DAT'...","{'APR': 1, 'NOV': 1, 'IMP': 0, 'CMP': 3, 'PNF'...",0.644999,0.671141,0.020833


In [32]:
data.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/review_hedge_score.csv',index=False)